In [72]:
import requests
import json
import os

filename = "./events.json"
if os.path.exists(filename):
    with open(filename, "r", encoding="utf-8") as f:
        try:
            existing_data = json.load(f)
        except json.JSONDecodeError:
            existing_data = []
else:
    existing_data = []

existing_ids = {event["id"] for event in existing_data if "id" in event}

returned = existing_data[:]

i = max(existing_ids) - 1 if existing_ids else 0
while 1:
    i += 1
    
    if i in existing_ids:
        print(f"Skipping {i}: Already exists")
        continue

    print(f"Fetching {i}: ", end="")
    
    res = requests.get(f'https://bestdori.com/api/events/{i}.json')
    
    print(res.status_code)

    if res.status_code == 404:
        break
    
    if res.status_code != 200:
        print(f"Skipping {i}: HTTP {res.status_code}")
        continue

    try:
        response = res.json()

        obj = {
            "id": i,
            "eventType": response.get('eventType'),
            "color": response.get('attributes', [{}])[0].get('attribute', 'Unknown'),
            "characters": [int(char.get('characterId')) for char in response.get('characters', [])]
        }

        print(f"{i}th event: ", obj)
        returned.append(obj)

    except json.JSONDecodeError:
        print(f"Skipping {i}: Invalid JSON response")
        continue

with open(filename, "w", encoding="utf-8") as f:
    json.dump(returned, f, indent=4, ensure_ascii=False)

print("Data successfully updated in events.json")


Skipping 294: Already exists
Fetching 295: 404
Data successfully updated in events.json


In [73]:
import json
import numpy as np
from collections import defaultdict

team_mapping = {
    '0': 'Mix',
    '1': 'PPP',
    '2': 'AG',
    '3': 'HHW',
    '4': 'PP',
    '5': 'R',
    '6': 'M',
    '7': 'RAS',
    '8': 'GO'
}

with open("events.json", "r", encoding="utf-8") as f:
    events = json.load(f)

character_teams = {i: (i - 1) // 5 + 1 for i in range(1, 41)}

event_types = [e["eventType"] for e in events]
colors = [e["color"] for e in events]
characters = [e["characters"] for e in events]

X = []

for i in range(len(event_types)):
    team_list = [character_teams[j] for j in characters[i]]
    team_index = team_list[0] if len(set(team_list)) == 1 else 0
    team = team_mapping[f'{team_index}']
    
    event_type = event_types[i]
    
    if event_type == 'festival' or event_type == 'story':
        event_type = 'others'
        
    X.append([event_type, colors[i], team])
    # X.append(["A", "A", team])

X = np.array(X)

n_val = len(X) * 10 // 100

diffs = defaultdict(list)
last_seen = {}

for idx, grp in enumerate(X[:, 2], start=1):
    if grp in last_seen:
        diffs[grp].append(idx - last_seen[grp])
    last_seen[grp] = idx

medians = {
    grp: (float(np.median(gaps)) if gaps else None)
    for grp, gaps in diffs.items()
}

valid = [m for m in medians.values() if m is not None]
gap_mean = int(np.round(np.mean(valid), 0)) if valid else None

X_print = X[-n_val:]

print(np.unique(X[:,0]))
print(np.unique(X[:,1]))
print(np.unique(X[:,2]))

['challenge' 'live_try' 'medley' 'mission_live' 'others' 'versus']
['cool' 'happy' 'powerful' 'pure']
['AG' 'GO' 'HHW' 'M' 'Mix' 'PP' 'PPP' 'R' 'RAS']


In [74]:
import tensorflow as tf

print(tf.config.list_physical_devices())

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [75]:
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras._tf_keras.keras.layers import LSTM, Dense, Embedding, Concatenate, Input, Dropout, Flatten
from keras._tf_keras.keras.models import Model
from keras._tf_keras.keras.losses import CategoricalCrossentropy
from keras._tf_keras.keras.metrics import CategoricalAccuracy
from keras._tf_keras.keras.optimizers import Adam

event_encoder = LabelEncoder()
color_encoder = LabelEncoder()
team_encoder = LabelEncoder()

event_encoder.fit(X[:, 0])                                                         # type: ignore
color_encoder.fit(X[:, 1])                                                                       # type: ignore                      
team_encoder.fit(X[:, 2])                                                                       # type: ignore

event_types_all = event_encoder.transform(X[:, 0])                                               # type: ignore
colors_all = color_encoder.transform(X[:, 1])                               # type: ignore
teams_all = team_encoder.transform(X[:, 2])                                                                                    # type: ignore

event_types = event_types_all[:-n_val]                                                          # type: ignore
colors = colors_all[:-n_val]                                            # type: ignore
teams = teams_all[:-n_val]                                                                                  # type: ignore

num_event_classes = len(event_encoder.classes_)
num_color_classes = len(color_encoder.classes_)
num_team_classes = len(team_encoder.classes_)

event_onehot = OneHotEncoder(sparse_output=False, categories='auto')
color_onehot = OneHotEncoder(sparse_output=False, categories='auto')
team_onehot = OneHotEncoder(sparse_output=False, categories='auto')

event_onehot.fit(event_types.reshape(-1, 1))                                             # type: ignore
color_onehot.fit(colors.reshape(-1, 1))                                              # type: ignore
team_onehot.fit(teams.reshape(-1, 1))                                                   # type: ignore

history_length = int(gap_mean)                                                                           # type: ignore

X_train = []
y_train_event = []
y_train_color = []
y_train_team = []

for i in range(len(event_types) - history_length):                                                      # type: ignore
    X_train.append(np.column_stack((
        event_types[i:i + history_length],                                                                    # type: ignore
        colors[i:i + history_length],                                                                    # type: ignore
        teams[i:i + history_length]                                                                        # type: ignore
    )))

    y_train_event.append(event_onehot.transform([[event_types[i + history_length]]])[0])                                                      # type: ignore
    y_train_color.append(color_onehot.transform([[colors[i + history_length]]])[0])                                                      # type: ignore
    y_train_team.append(team_onehot.transform([[teams[i + history_length]]])[0])                                                      # type: ignore

X_train = np.array(X_train)
y_train_event = np.array(y_train_event)
y_train_color = np.array(y_train_color)
y_train_team = np.array(y_train_team)

X_train_event = X_train[:, :, 0]
X_train_color = X_train[:, :, 1]
X_train_team = X_train[:, :, 2]

event_input = Input(shape=(history_length,))
color_input = Input(shape=(history_length,))
team_input = Input(shape=(history_length,))

event_embed = Embedding(input_dim=num_event_classes, output_dim=4)(event_input)
color_embed = Embedding(input_dim=num_color_classes, output_dim=4)(color_input)
team_embed = Embedding(input_dim=num_team_classes, output_dim=4)(team_input)

merged = Concatenate(axis=-1)([event_embed, color_embed, team_embed])

main_model = LSTM(48, activation='relu', recurrent_dropout=.35, dropout=.35)(merged)
main_model = Dense(16)(main_model)

event_output = Dense(num_event_classes, activation='softmax', name="event_output")(main_model)
color_output = Dense(num_color_classes, activation='softmax', name="color_output")(main_model)
team_output = Dense(num_team_classes, activation='softmax', name="team_output")(main_model)

opt = Adam()
model = Model(inputs=[event_input, color_input, team_input], 
              outputs=[event_output, color_output, team_output])
model.compile(optimizer=opt,                                                            # type: ignore
              loss={'event_output': CategoricalCrossentropy(), 
                    'color_output': CategoricalCrossentropy(), 
                    'team_output': CategoricalCrossentropy()},
              metrics=[CategoricalAccuracy(), 
                       CategoricalAccuracy(), 
                       CategoricalAccuracy()])

model.summary()


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_24      │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_25      │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_26      │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_24        │ (None, 9, 4)      │         24 │ input_layer_24[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_25        │ (None, 9, 4)      │         16 │ input_layer_25[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_26        │ (None, 9, 4)      │         36 │ input_layer_26[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_8       │ (None, 9, 12)     │          0 │ embedding_24[0][… │
│ (Concatenate)       │                   │            │ embedding_25[0][… │
│                     │                   │            │ embedding_26[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_8 (LSTM)       │ (None, 48)        │     11,712 │ concatenate_8[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 16)        │        784 │ lstm_8[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ event_output        │ (None, 6)         │        102 │ dense_8[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ color_output        │ (None, 4)         │         68 │ dense_8[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ team_output (Dense) │ (None, 9)         │        153 │ dense_8[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,895 (50.37 KB)

 Trainable params: 12,895 (50.37 KB)

 Non-trainable params: 0 (0.00 B)

In [76]:
epp = 1024

import matplotlib.pyplot as plt
import os
from keras._tf_keras.keras.callbacks import EarlyStopping, Callback

class TerminateOnAverageBaseline(Callback):
    def __init__(self, metrics=None, baseline=0.8):
        super().__init__()
        self.metrics   = metrics or [
            "event_output_categorical_accuracy",
            "color_output_categorical_accuracy",
            "team_output_categorical_accuracy"
        ]
        self.baseline  = baseline

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        vals = []
        for m in self.metrics:
            v = logs.get(m)
            if v is None:
                v = logs.get(m.replace("val_", ""))  
            if v is None:
                print(f"Warning: metric '{m}' not found in logs; skipping.")
            else:
                vals.append(v)

        if not vals:
            return

        avg = sum(vals) / len(vals)
        if avg >= self.baseline:
            print(
                f"\nEpoch {epoch+1}: average = "
                f"{avg:.4f} ≥ {self.baseline:.4f}; stopping training."
            )
            self.model.stop_training = True

es_loss = EarlyStopping(
    monitor='loss',
    patience=16,
    start_from_epoch=epp//10,
    restore_best_weights=True
)

if not os.path.exists("./dori_pred_lstm.keras"):
    target_acc = .8

    history = model.fit([X_train_event, X_train_color, X_train_team], 
            {'event_output': y_train_event, 'color_output': y_train_color, 'team_output': y_train_team}, 
            epochs=epp, verbose=2, callbacks=[
                es_loss,
                TerminateOnAverageBaseline(baseline=target_acc),
                TerminateOnAverageBaseline(metrics=['event_output_categorical_accuracy'], baseline=target_acc * 1.1),
                TerminateOnAverageBaseline(metrics=['color_output_categorical_accuracy'], baseline=target_acc * 1.1),
                TerminateOnAverageBaseline(metrics=['team_output_categorical_accuracy'], baseline=target_acc * 1.1),
                ])
            # epochs=epp, verbose=2)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Total Loss')
    plt.plot(history.history['event_output_loss'], label='Event Loss')
    plt.plot(history.history['color_output_loss'], label='Color Loss')
    plt.plot(history.history['team_output_loss'], label='Team Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch (LSTM)')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['event_output_categorical_accuracy'], label='Event Accuracy')
    plt.plot(history.history['color_output_categorical_accuracy'], label='Color Accuracy')
    plt.plot(history.history['team_output_categorical_accuracy'], label='Team Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch (LSTM)')
    plt.legend()

    plt.tight_layout()
    plt.show()


In [77]:
from keras._tf_keras.keras.models import load_model
import numpy as np

model = load_model("./dori_pred_lstm.keras")

model.summary()


Model: "functional_46"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_138     │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_139     │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_140     │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_138       │ (None, 9, 4)      │         24 │ input_layer_138[… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_139       │ (None, 9, 4)      │         16 │ input_layer_139[… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_140       │ (None, 9, 4)      │         36 │ input_layer_140[… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_46      │ (None, 9, 12)     │          0 │ embedding_138[0]… │
│ (Concatenate)       │                   │            │ embedding_139[0]… │
│                     │                   │            │ embedding_140[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_46 (LSTM)      │ (None, 48)        │     11,712 │ concatenate_46[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_46 (Dense)    │ (None, 16)        │        784 │ lstm_46[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ event_output        │ (None, 6)         │        102 │ dense_46[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attr_output (Dense) │ (None, 4)         │         68 │ dense_46[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ team_output (Dense) │ (None, 9)         │        153 │ dense_46[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 38,687 (151.12 KB)

 Trainable params: 12,895 (50.37 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 25,792 (100.75 KB)

In [78]:

val_start = len(event_types)
val_end = len(event_types_all)
val_window = val_end - val_start

pred_events = []
pred_colors  = []
pred_teams  = []

for idx in range(val_start, val_end):
    seq_e = event_types_all[idx - history_length:idx]
    seq_a = colors_all[idx - history_length:idx]
    seq_t = teams_all[idx - history_length:idx]

    inp_e = np.expand_dims(seq_e, 0)
    inp_a = np.expand_dims(seq_a, 0)
    inp_t = np.expand_dims(seq_t, 0)

    pe, pa, pt = model.predict([inp_e, inp_a, inp_t], verbose=0)

    i_e = np.argmax(pe, axis=-1)[0]
    i_a = np.argmax(pa, axis=-1)[0]
    i_t = np.argmax(pt, axis=-1)[0]

    pred_events.append(event_encoder.inverse_transform([i_e])[0])
    pred_colors.append(color_encoder.inverse_transform([i_a])[0])
    pred_teams.append(team_encoder.inverse_transform([i_t])[0])
    
actual_events = X[val_start:val_end, 0]
actual_colors  = X[val_start:val_end, 1]
actual_teams  = X[val_start:val_end, 2]

GREEN = '\033[92m'
RED = '\033[91m'
ENDC = '\033[0m'

w_e, w_a, w_t = 12, 8, 5

print("-" * 80)
for i in range(val_window):
    idx = len(X) - n_val + i

    a_e = actual_events[i]
    a_a = actual_colors[i]
    a_t = actual_teams[i]

    p_e = pred_events[i]
    p_a = pred_colors[i]
    p_t = pred_teams[i]

    a_e_pad = a_e.ljust(w_e)
    a_a_pad = a_a.ljust(w_a)
    a_t_pad = a_t.ljust(w_t)

    p_e_pad = p_e.ljust(w_e)
    p_a_pad = p_a.ljust(w_a)
    p_t_pad = p_t.ljust(w_t)

    pe_col = f"{GREEN if p_e==a_e else RED}{p_e_pad}{ENDC}"
    pa_col = f"{GREEN if p_a==a_a else RED}{p_a_pad}{ENDC}"
    pt_col = f"{GREEN if p_t==a_t else RED}{p_t_pad}{ENDC}"

    print(
        f"{idx+1:3d}) Actual ["
        f"{a_e_pad} {a_a_pad} {a_t_pad}]"
        f" | Pred   ["
        f"{pe_col} {pa_col} {pt_col}]"
    )
print("-" * 80)

evt_acc = np.mean([p == a for p, a in zip(pred_events, actual_events)])
att_acc = np.mean([p == a for p, a in zip(pred_colors, actual_colors)])
tm_acc = np.mean([p == a for p, a in zip(pred_teams, actual_teams)])
joint_acc = np.mean([
    (pe == ae) and (pa == aa) and (pt == at)
    for pe, ae, pa, aa, pt, at in zip(
        pred_events, actual_events, 
        pred_colors, actual_colors,
        pred_teams, actual_teams
    )
])

any_acc = np.mean([
    (pe == ae) or (pa == aa) or (pt == at)
    for pe, ae, pa, aa, pt, at in zip(
        pred_events, actual_events,
        pred_colors, actual_colors,
        pred_teams, actual_teams
    )
])

two_acc = np.mean([
    sum([pe == ae, pa == aa, pt == at]) >= 2
    for pe, ae, pa, aa, pt, at in zip(
        pred_events,    actual_events,
        pred_colors,     actual_colors,
        pred_teams,     actual_teams
    )
])

avg = (evt_acc * num_event_classes + att_acc * num_color_classes + tm_acc * num_team_classes) / (num_event_classes + num_color_classes + num_team_classes)

print(f"Accuracy over {val_window} unseen data predicted using LSTM\n"
      f"Event: {evt_acc:.2%}, "
      f"Color: {att_acc:.2%}, "
      f"Team: {tm_acc:.2%}, "
      f"WEIGHTED-AVG: {avg:.2%}\n"
      f"AT-LEAST-ONE: {any_acc:.2%}, "
      f"AT-LEAST-TWO: {two_acc:.2%}, "
      f"ALL-THREE: {joint_acc:.2%}")
print("-" * 80)


--------------------------------------------------------------------------------
266) Actual [challenge    happy    Mix  ] | Pred   [mission_live pure     Mix  ]
267) Actual [mission_live pure     PP   ] | Pred   [mission_live pure     PP   ]
268) Actual [live_try     powerful AG   ] | Pred   [challenge    cool     AG   ]
269) Actual [versus       cool     RAS  ] | Pred   [mission_live cool     PPP  ]
270) Actual [challenge    pure     PPP  ] | Pred   [mission_live happy    PP   ]
271) Actual [mission_live happy    Mix  ] | Pred   [mission_live cool     HHW  ]
272) Actual [medley       powerful M    ] | Pred   [versus       powerful Mix  ]
273) Actual [mission_live pure     R    ] | Pred   [versus       pure     Mix  ]
274) Actual [live_try     powerful Mix  ] | Pred   [versus       cool     Mix  ]
275) Actual [versus       cool     HHW  ] | Pred   [versus       cool     AG   ]
276) Actual [mission_live pure     PPP  ] | Pred   [mission_live powerful PP   ]
277) Actual [live_try     ha

In [79]:

val_start = len(event_types)
val_end = len(event_types_all)
val_window = val_end - val_start

pred_events = []
pred_colors  = []
pred_teams  = []

for idx in range(val_start, val_end):
    seq_e = event_types_all[idx - history_length : idx]
    seq_a = colors_all[idx - history_length : idx]
    seq_t = teams_all[idx - history_length : idx]

    inp_e = np.expand_dims(seq_e, 0)
    inp_a = np.expand_dims(seq_a, 0)
    inp_t = np.expand_dims(seq_t, 0)

    pe, pa, pt = model.predict([inp_e, inp_a, inp_t], verbose=0)

    top2_e = np.argsort(pe, axis=-1)[0][::-1][:2]
    if top2_e[0] != seq_e[-1]:
        chosen_e = top2_e[0]
    else:
        chosen_e = top2_e[1]

    top2_a = np.argsort(pa, axis=-1)[0][::-1][:2]
    if top2_a[0] != seq_a[-1]:
        chosen_a = top2_a[0]
    else:
        chosen_a = top2_a[1]

    top2_t = np.argsort(pt, axis=-1)[0][::-1][:2]
    if top2_t[0] != seq_t[-1]:
        chosen_t = top2_t[0]
    else:
        chosen_t = top2_t[1]

    pred_events.append(event_encoder.inverse_transform([chosen_e])[0])
    pred_colors .append(color_encoder.inverse_transform([chosen_a])[0])
    pred_teams .append(team_encoder.inverse_transform([chosen_t])[0])

actual_events = X[val_start:val_end, 0]
actual_colors  = X[val_start:val_end, 1]
actual_teams  = X[val_start:val_end, 2]

GREEN = '\033[92m'
RED = '\033[91m'
ENDC = '\033[0m'

w_e, w_a, w_t = 12, 8, 5

print("-" * 80)
for i in range(val_window):
    idx = len(X) - n_val + i

    a_e = actual_events[i]
    a_a = actual_colors[i]
    a_t = actual_teams[i]

    p_e = pred_events[i]
    p_a = pred_colors[i]
    p_t = pred_teams[i]

    a_e_pad = a_e.ljust(w_e)
    a_a_pad = a_a.ljust(w_a)
    a_t_pad = a_t.ljust(w_t)

    p_e_pad = p_e.ljust(w_e)
    p_a_pad = p_a.ljust(w_a)
    p_t_pad = p_t.ljust(w_t)

    pe_col = f"{GREEN if p_e==a_e else RED}{p_e_pad}{ENDC}"
    pa_col = f"{GREEN if p_a==a_a else RED}{p_a_pad}{ENDC}"
    pt_col = f"{GREEN if p_t==a_t else RED}{p_t_pad}{ENDC}"

    print(
        f"{idx+1:3d}) Actual ["
        f"{a_e_pad} {a_a_pad} {a_t_pad}]"
        f" | Pred   ["
        f"{pe_col} {pa_col} {pt_col}]"
    )
print("-" * 80)

evt_acc = np.mean([p == a for p, a in zip(pred_events, actual_events)])
att_acc = np.mean([p == a for p, a in zip(pred_colors, actual_colors)])
tm_acc = np.mean([p == a for p, a in zip(pred_teams, actual_teams)])
joint_acc = np.mean([
    (pe == ae) and (pa == aa) and (pt == at)
    for pe, ae, pa, aa, pt, at in zip(
        pred_events, actual_events, 
        pred_colors, actual_colors,
        pred_teams, actual_teams
    )
])

any_acc = np.mean([
    (pe == ae) or (pa == aa) or (pt == at)
    for pe, ae, pa, aa, pt, at in zip(
        pred_events, actual_events,
        pred_colors, actual_colors,
        pred_teams, actual_teams
    )
])

two_acc = np.mean([
    sum([pe == ae, pa == aa, pt == at]) >= 2
    for pe, ae, pa, aa, pt, at in zip(
        pred_events,    actual_events,
        pred_colors,     actual_colors,
        pred_teams,     actual_teams
    )
])

avg = (evt_acc * num_event_classes + att_acc * num_color_classes + tm_acc * num_team_classes) / (num_event_classes + num_color_classes + num_team_classes)

print(f"Accuracy over {val_window} unseen data predicted using LSTM + HIH\n"
      f"Event: {evt_acc:.2%}, "
      f"Color: {att_acc:.2%}, "
      f"Team: {tm_acc:.2%}, "
      f"WEIGHTED-AVG: {avg:.2%}\n"
      f"AT-LEAST-ONE: {any_acc:.2%}, "
      f"AT-LEAST-TWO: {two_acc:.2%}, "
      f"ALL-THREE: {joint_acc:.2%}")
print("-" * 80)


--------------------------------------------------------------------------------
266) Actual [challenge    happy    Mix  ] | Pred   [mission_live pure     Mix  ]
267) Actual [mission_live pure     PP   ] | Pred   [mission_live pure     PP   ]
268) Actual [live_try     powerful AG   ] | Pred   [challenge    cool     AG   ]
269) Actual [versus       cool     RAS  ] | Pred   [mission_live cool     PPP  ]
270) Actual [challenge    pure     PPP  ] | Pred   [mission_live happy    PP   ]
271) Actual [mission_live happy    Mix  ] | Pred   [mission_live cool     HHW  ]
272) Actual [medley       powerful M    ] | Pred   [versus       powerful M    ]
273) Actual [mission_live pure     R    ] | Pred   [versus       pure     Mix  ]
274) Actual [live_try     powerful Mix  ] | Pred   [versus       cool     Mix  ]
275) Actual [versus       cool     HHW  ] | Pred   [versus       cool     AG   ]
276) Actual [mission_live pure     PPP  ] | Pred   [mission_live powerful PP   ]
277) Actual [live_try     ha

In [80]:
last_seq_event = event_types_all[-history_length - 1:]
last_seq_color = colors_all[-history_length - 1:]
last_seq_team = teams_all[-history_length - 1:]

last_seq_event = np.expand_dims(last_seq_event, axis=0)
last_seq_color = np.expand_dims(last_seq_color, axis=0)
last_seq_team = np.expand_dims(last_seq_team, axis=0)

next_event_pred, next_color_pred, next_team_pred = model.predict([last_seq_event, last_seq_color, last_seq_team], verbose=0)

top_k = 5

top_events = np.argsort(next_event_pred, axis=-1)[:, -top_k:][:, ::-1]
top_events_prob = np.sort(next_event_pred, axis=-1)[:, -top_k:][:, ::-1] * 100

top_color = np.argsort(next_color_pred, axis=-1)[:, -top_k:][:, ::-1]
top_color_prob = np.sort(next_color_pred, axis=-1)[:, -top_k:][:, ::-1] * 100

top_teams = np.argsort(next_team_pred, axis=-1)[:, -top_k:][:, ::-1]
top_teams_prob = np.sort(next_team_pred, axis=-1)[:, -top_k:][:, ::-1] * 100

predicted_events = [event_encoder.inverse_transform(sample) for sample in top_events]
predicted_colors = [color_encoder.inverse_transform(sample) for sample in top_color]
predicted_teams = [team_encoder.inverse_transform(sample) for sample in top_teams]

print("-" * 75)
print(f"Predicted Next Most Likely Information:\n")
print(f"Type\t{predicted_events[0]}\nP (%)\t{[f'{p:.2f}' for p in top_events_prob[0]]}\n")
print(f"Color\t{predicted_colors[0]}\nP (%)\t{[f'{p:.2f}' for p in top_color_prob[0]]}\n")
print(f"Team\t{predicted_teams[0]}\nP (%)\t{[f'{p:.2f}' for p in top_teams_prob[0]]}")
print("-" * 75)
print("Most recent 15 events:\n")
for i in range(len(X_print) if len(X_print) <= 15 else 15):
    print(f"{len(X) - i}:", X_print[::-1][i])
print("-" * 75)

---------------------------------------------------------------------------
Predicted Next Most Likely Information:

Type	['mission_live' 'challenge' 'live_try' 'versus' 'medley']
P (%)	['36.68', '17.93', '16.40', '14.66', '8.10']

Color	['powerful' 'pure' 'cool' 'happy']
P (%)	['60.00', '29.82', '8.16', '2.02']

Team	['Mix' 'M' 'PPP' 'R' 'RAS']
P (%)	['56.75', '40.11', '0.90', '0.90', '0.85']
---------------------------------------------------------------------------
Most recent 15 events:

294: ['medley' 'happy' 'AG']
293: ['challenge' 'cool' 'M']
292: ['live_try' 'pure' 'HHW']
291: ['mission_live' 'powerful' 'R']
290: ['versus' 'cool' 'PP']
289: ['challenge' 'pure' 'Mix']
288: ['others' 'powerful' 'RAS']
287: ['medley' 'cool' 'PPP']
286: ['mission_live' 'happy' 'GO']
285: ['live_try' 'cool' 'AG']
284: ['challenge' 'happy' 'R']
283: ['mission_live' 'pure' 'HHW']
282: ['live_try' 'cool' 'M']
281: ['challenge' 'powerful' 'Mix']
280: ['live_try' 'happy' 'RAS']
--------------------------